# Mini Project to implement CNN object detection on any data. Discuss numerous performance
evaluation metrics for evaluating the object detecting algorithms' performance, Take outputs as a
comparative results of algorithms.


In [1]:
import numpy as np
import argparse  #argument parameter
import time
import cv2     #opencv computer vision library
import os

# construct the argument parse and parse the arguments

In [2]:
import argparse
import sys

try:
    get_ipython              #exception handling 
    in_ipython = True
except NameError:
    in_ipython = False


default_image_path = "img.jpg"
default_confidence_threshold = 0.5  # probability exection 
default_nms_threshold = 0.3         # parameter threshold 

if in_ipython:                                    
    image_path = default_image_path                              #to check data is it present in environment or not
    confidence_threshold = default_confidence_threshold
    nms_threshold = default_nms_threshold
else:
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--image", default=default_image_path,
        help="path to input image")
    ap.add_argument("-c", "--confidence", type=float, default=default_confidence_threshold,
        help="minimum probability to filter weak detections, IoU threshold")
    ap.add_argument("-t", "--threshold", type=float, default=default_nms_threshold,
        help="threshold when applying non-maxima suppression")
    args = vars(ap.parse_args())

    image_path = args["image"]
    confidence_threshold = args["confidence"]
    nms_threshold = args["threshold"]

print("Image Path:", image_path)
print("Confidence Threshold:", confidence_threshold)
print("NMS Threshold:", nms_threshold)


Image Path: img.jpg
Confidence Threshold: 0.5
NMS Threshold: 0.3


# load the COCO class labels our YOLO model was trained on

In [3]:
#file loading
labelsPath = 'coco.names'
LABELS = open(labelsPath).read().strip().split("\n")


# initialize a list of colors to represent each possible class label

In [4]:
#Initialize color
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

# paths to the YOLO weights and model configuration

In [5]:
weightsPath = 'yolov3 (1).weights'
configPath = 'yolov3.cfg'


# load our YOLO object detector trained on COCO dataset (80 classes)

In [6]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# load our input image and grab its spatial dimensions

In [7]:
image = cv2.imread(image_path) #to show image dimension 
if image is None:
    print("Error: Unable to read image at", image_path)
else:
    (H, W) = image.shape[:2]


# determine only the *output* layer names that we need from YOLO

In [8]:
import cv2

# Load the YOLO model
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

#  names of the output layers
output_layer_indices = net.getUnconnectedOutLayers()

# names of all layers in the network
layer_names = net.getLayerNames()

# names of the output layers using the indices directly
output_layer_names = [layer_names[i - 1] for i in output_layer_indices]

print("Output Layer Names:", output_layer_names)



Output Layer Names: ['yolo_82', 'yolo_94', 'yolo_106']


# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities

In [9]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),   #forward pass
                              swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(output_layer_names)  


# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively

In [10]:

boxes = []
confidences = []
classIDs = []


# loop over each of the layer outputs

In [11]:

args = {
    "confidence": 0.5  
}

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        print("Scores Shape:", scores.shape)
        print("Scores Content:", scores)

        if classID < len(scores):
            confidence = scores[classID]
            
           
            if confidence > args["confidence"]:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
        else:
            print("ClassID is out of bounds:", classID)


Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Scores Shape: (80,)
Scores Content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

# apply non-maxima suppression to suppress weak, overlapping bounding boxes

In [12]:
ap = argparse.ArgumentParser()

ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections, IoU threshold")
ap.add_argument("-t", "--threshold", type=float, default=0.3,
	help="threshold when applying non-maxima suppression")

args, unknown = ap.parse_known_args()

idxs = cv2.dnn.NMSBoxes(boxes, confidences, args.confidence, args.threshold)


# ensure at least one detection exists

In [13]:

if len(idxs) > 0:
	for i in idxs.flatten():
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])

		color = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, color, 2)


# show the output image

In [14]:
cv2.imshow("Image", image)
cv2.waitKey(0)

-1